In [14]:
import pandas as pd
import re
import mysql.connector

In [34]:
df = pd.read_csv('../data/blaine_weather.csv')

In [35]:
df.columns = [re.sub(r'^ ', '', val) for val in df.columns.values]  # extra spaces removed in column names

In [37]:
df.Events = df.Events.fillna('Clear')

In [38]:
df["Events"].value_counts()

Rain                 175
Clear                161
Fog                   39
Fog-Rain              10
Rain-Snow              7
Fog-Rain-Snow          3
Snow                   2
Rain-Thunderstorm      1
Name: Events, dtype: int64

In [39]:
df.PST = df.PST.str.replace(r'-(\d)-(\d)$', '-0\g<1>-0\g<2>')
df.PST = df.PST.str.replace(r'-(\d)-(\d\d)$', '-0\g<1>-\g<2>')
df.PST = df.PST.str.replace(r'-(\d\d)-(\d)$', '-\g<1>-0\g<2>')

In [41]:
df.PST = pd.to_datetime(df.PST, format='%Y-%m-%d')

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 398 entries, 0 to 397
Data columns (total 23 columns):
PST                          398 non-null datetime64[ns]
Max TemperatureF             398 non-null int64
Mean TemperatureF            398 non-null int64
Min TemperatureF             398 non-null int64
Max Dew PointF               398 non-null int64
MeanDew PointF               398 non-null int64
Min DewpointF                398 non-null int64
Max Humidity                 398 non-null int64
Mean Humidity                398 non-null int64
Min Humidity                 398 non-null int64
Max Sea Level PressureIn     398 non-null float64
Mean Sea Level PressureIn    398 non-null float64
Min Sea Level PressureIn     398 non-null float64
Max VisibilityMiles          398 non-null int64
Mean VisibilityMiles         398 non-null int64
Min VisibilityMiles          398 non-null int64
Max Wind SpeedMPH            398 non-null int64
Mean Wind SpeedMPH           398 non-null int64
Max Gust SpeedMP

In [43]:
df.head()

,PST,Max TemperatureF,Mean TemperatureF,Min TemperatureF,Max Dew PointF,MeanDew PointF,Min DewpointF,Max Humidity,Mean Humidity,Min Humidity,...,Max VisibilityMiles,Mean VisibilityMiles,Min VisibilityMiles,Max Wind SpeedMPH,Mean Wind SpeedMPH,Max Gust SpeedMPH,PrecipitationIn,CloudCover,Events,WindDirDegrees
0,2006-01-01,53,48,43,42,38,35,86,70,52,...,10,10,9,30,13,48,0.01,5,Rain,134
1,2006-01-02,48,42,37,43,40,36,96,83,66,...,10,10,9,14,8,20,0.04,7,Rain,180
2,2006-01-03,45,40,35,39,36,34,96,80,68,...,10,10,10,16,8,23,0.02,6,Rain,127
3,2006-01-04,51,47,43,40,38,36,86,73,56,...,10,10,10,16,11,NaN,0.00,5,Rain,112
4,2006-01-05,60,52,44,48,44,34,100,83,39,...,10,8,2,26,11,39,1.06,8,Rain,118


In [46]:
df.columns.values

array(['PST', 'Max TemperatureF', 'Mean TemperatureF', 'Min TemperatureF',
       'Max Dew PointF', 'MeanDew PointF', 'Min DewpointF', 'Max Humidity',
       'Mean Humidity', 'Min Humidity', 'Max Sea Level PressureIn',
       'Mean Sea Level PressureIn', 'Min Sea Level PressureIn',
       'Max VisibilityMiles', 'Mean VisibilityMiles',
       'Min VisibilityMiles', 'Max Wind SpeedMPH', 'Mean Wind SpeedMPH',
       'Max Gust SpeedMPH', 'PrecipitationIn', 'CloudCover', 'Events',
       'WindDirDegrees'], dtype=object)

In [48]:
data = np.array(df[['PST', 'Max TemperatureF', 'Mean TemperatureF', 'Min TemperatureF',
       'Max VisibilityMiles', 'Mean VisibilityMiles',
       'Min VisibilityMiles', 'Max Wind SpeedMPH', 'Mean Wind SpeedMPH',
       'Max Gust SpeedMPH', 'PrecipitationIn', 'Events']])

In [50]:
data

array([[Timestamp('2006-01-01 00:00:00'), 53, 48, ..., 48.0, 0.01, 'Rain'],
       [Timestamp('2006-01-02 00:00:00'), 48, 42, ..., 20.0, 0.04, 'Rain'],
       [Timestamp('2006-01-03 00:00:00'), 45, 40, ..., 23.0, 0.02, 'Rain'],
       ..., 
       [Timestamp('2007-01-31 00:00:00'), 48, 38, ..., nan, 0.0, 'Fog'],
       [Timestamp('2007-02-01 00:00:00'), 52, 42, ..., 24.0, 0.0, 'Clear'],
       [Timestamp('2007-02-02 00:00:00'), 44, 34, ..., nan, 0.0, 'Clear']], dtype=object)

In [52]:
conn = mysql.connector.connect(user='dev', password='dev', host='localhost', database='border')
cur = conn.cursor()

query = '''
        INSERT INTO weather 
            (date, temp_max, temp_mean, temp_min, viz_max, viz_mean, viz_min, wind_max, wind_mean, wind_gust,
             precip, events)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        '''

cur.execute(query, data[0])
conn.commit()

cur.close()
conn.close()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)' at line 4

## Easier to import CSV directly into MySQL